In [5]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
%matplotlib inline

## Reading the Covid, Zika and Dengue datasets here

In [12]:
covid_data = pd.read_csv('Datasets/covid_19_data.csv.gz')
dengue_data = pd.read_csv('Datasets/dengue_data.csv.gz')
zika_data = pd.read_csv('zika/zika16to18.csv')
malaria_data = pd.read_csv('Datasets/malaria_data.csv.gz')


zika_data.columns = ['_'.join(c.lower().split(' ')) for c in zika_data.columns]
zika_data = zika_data.rename(columns= {'noncumulative_confirmed': 'no_cases',
                                       'noncumulative_zika_case_deaths': 'no_deaths',
                                       'confirmed': 'cumulative_cases',
                                       'zika_case_deaths': 'cumulative_deaths',
                                       'country/territory': 'country'})

dengue_data['date'] = pd.to_datetime(dengue_data.date)
covid_data['date'] = pd.to_datetime(covid_data.date)
zika_data['date'] = pd.to_datetime(zika_data.date)
malaria_data['date'] = pd.to_datetime(malaria_data.date)

### Adding the disease and transmission_mode columns

In [26]:
covid_data['disease'] = 'covid-19'
dengue_data['disease'] = 'dengue'
zika_data['disease'] = 'zika'
malaria_data['disease']='malaria'

covid_data['transmission_mode'] = 'air'
dengue_data['transmission_mode'] = 'mosquito'
zika_data['transmission_mode'] = 'mosquito'
malaria_data['transmission_mode']='mosquito'


### Checking if the must columns are present in the datasets are not

In [30]:
must_columns = ['country','date','year','month','day','transmission_mode',
                'no_cases','no_deaths','cumulative_cases','cumulative_deaths','disease']

assert pd.Series(must_columns).isin(covid_data.columns).all()
assert pd.Series(must_columns).isin(dengue_data.columns).all()
assert pd.Series(must_columns).isin(zika_data.columns).all()
assert pd.Series(must_columns).isin(malaria_data.columns).all()

In [31]:
# may be doing with the week number 
# clean up the country column
# Merge the malaria data
    ## Instead of the spikes by country and we just cateorize it by the continent - that way we only have the 7 thingst to  manupulate
    
# weekly counts is safer

In [32]:
df = pd.concat([covid_data[must_columns],
                zika_data[must_columns],
                dengue_data[must_columns],
               malaria_data[must_columns]])

df['week'] = df.date.dt.week

In [33]:
df.head()

,country,date,year,month,day,transmission_mode,no_cases,no_deaths,cumulative_cases,cumulative_deaths,disease,week
0,Afghanistan,2020-01-22,2020,1,22,air,0.0,0.0,0.0,0.0,covid-19,4
1,Afghanistan,2020-01-23,2020,1,23,air,0.0,0.0,0.0,0.0,covid-19,4
2,Afghanistan,2020-01-24,2020,1,24,air,0.0,0.0,0.0,0.0,covid-19,4
3,Afghanistan,2020-01-25,2020,1,25,air,0.0,0.0,0.0,0.0,covid-19,4
4,Afghanistan,2020-01-26,2020,1,26,air,0.0,0.0,0.0,0.0,covid-19,4
